<a href="https://colab.research.google.com/github/AbhinayReddyYarva/NLP-SpamSMSDetection-KaggleDataset/blob/master/ModelBuildEvaluateTPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Building ML Classifiers: Basic Random Forest & Gradient Boosting models (Running in Google colab with TPU)

### Reading & cleaning text

In [0]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import string
import time
import datetime


nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation]) # Counting puntuations
    return round(count/(len(text) - text.count(" ")), 3)*100  # Percentage of punctuations

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation]) # removing puntuations and creating list
    tokens = re.split('\W+', text) # removing multiple special characters --> '\W+' ('\W' -> single special character)
    #tokens = re.findall('\w+', text)  # alternative to above statement (Split -> '\W+'; findall -> '\w+')
    text = [ps.stem(word) for word in tokens if word not in stopwords]  # stemmig the words
    return text

# TF-IDF
tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])
X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
print(X_features.head())

# CountVectorizer
count_vect = CountVectorizer(analyzer=clean_text)
X_count = count_vect.fit_transform(data['body_text'])
X_count_feat = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_count.toarray())], axis=1)
print(X_count_feat.head())

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
   body_len  punct%    0    1    2    3  ...  8098  8099  8100  8101  8102  8103
0       128     4.7  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
1        49     4.1  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
2        62     3.2  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
3        28     7.1  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0
4       135     4.4  0.0  0.0  0.0  0.0  ...   0.0   0.0   0.0   0.0   0.0   0.0

[5 rows x 8106 columns]
   body_len  punct%  0  1  2  3  4  ...  8097  8098  8099  8100  8101  8102  8103
0       128     4.7  0  0  0  0  0  ...     0     0     0     0     0     0     0
1        49     4.1  0  0  0  0  0  ...     0     0     0     0     0     0     0
2        62     3.2  0  0  0  0  0  ...     0     0     0     0     0     0     0
3        28     7.1  0  0  0  0  0  ...   

### Exploring RandomForestClassifier Attributes & Hyperparameters

In [0]:
from sklearn.ensemble import RandomForestClassifier

In [0]:
print(dir(RandomForestClassifier))
print(RandomForestClassifier())

['__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_estimator_type', '_get_param_names', '_get_tags', '_make_estimator', '_more_tags', '_required_parameters', '_set_oob_score', '_validate_X_predict', '_validate_estimator', '_validate_y_class_weight', 'apply', 'decision_path', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score', 'set_params']
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto

### Exploing RandomForestClassifier through Cross-Validation

In [0]:
from sklearn.model_selection import KFold, cross_val_score

In [0]:
rf_kfold = RandomForestClassifier(n_jobs=-1)
k_fold = KFold(n_splits=5)
cross_val_score(rf_kfold, X_features, data['label'], cv=k_fold, scoring='accuracy', n_jobs=-1)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


array([0.97666068, 0.97307002, 0.9703504 , 0.95956873, 0.96495957])

### Exploring RandomForestClassifier through holdout set

In [0]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data['label'], test_size=0.2)

In [0]:
rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)

In [0]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)[0:10]

[(0.07978630579894755, 'body_len'),
 (0.045006394742685245, 7350),
 (0.03109324868089661, 2031),
 (0.028339612098920906, 3134),
 (0.02545401438726829, 4796),
 (0.0239016969778129, 6285),
 (0.017262458664456578, 5988),
 (0.01702123825699941, 397),
 (0.01573807087961601, 392),
 (0.015015466919174561, 1803)]

In [0]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [0]:
print('Precision: {} / Recall: {} / Accuracy: {}'.format(round(precision, 3),
                                                        round(recall, 3),
                                                        round((y_pred==y_test).sum() / len(y_pred),3)))

Precision: 1.0 / Recall: 0.579 / Accuracy: 0.94


### Building a Grid Search

In [0]:
def train_RF(n_est, depth):
    rf1 = RandomForestClassifier(n_estimators=n_est, max_depth=depth, n_jobs=-1)
    rf_model1 = rf.fit(X_train, y_train)
    y_pred1 = rf_model.predict(X_test)
    precision1, recall1, fscore1, support1 = score(y_test, y_pred1, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        n_est, depth, round(precision1, 3), round(recall1, 3),
        round((y_pred1==y_test).sum() / len(y_pred1), 3)))

In [0]:
print(datetime.datetime.now())
for n_est in [10, 50, 100]:
    for depth in [10, 20, 30, None]:
        train_RF(n_est, depth)
print(datetime.datetime.now())

2019-06-05 17:40:22.419729
Est: 10 / Depth: 10 ---- Precision: 1.0 / Recall: 0.535 / Accuracy: 0.934
Est: 10 / Depth: 20 ---- Precision: 1.0 / Recall: 0.566 / Accuracy: 0.938
Est: 10 / Depth: 30 ---- Precision: 1.0 / Recall: 0.541 / Accuracy: 0.934
Est: 10 / Depth: None ---- Precision: 1.0 / Recall: 0.585 / Accuracy: 0.941
Est: 50 / Depth: 10 ---- Precision: 1.0 / Recall: 0.566 / Accuracy: 0.938
Est: 50 / Depth: 20 ---- Precision: 1.0 / Recall: 0.535 / Accuracy: 0.934
Est: 50 / Depth: 30 ---- Precision: 1.0 / Recall: 0.56 / Accuracy: 0.937
Est: 50 / Depth: None ---- Precision: 1.0 / Recall: 0.566 / Accuracy: 0.938
Est: 100 / Depth: 10 ---- Precision: 1.0 / Recall: 0.566 / Accuracy: 0.938
Est: 100 / Depth: 20 ---- Precision: 1.0 / Recall: 0.541 / Accuracy: 0.934
Est: 100 / Depth: 30 ---- Precision: 1.0 / Recall: 0.541 / Accuracy: 0.934
Est: 100 / Depth: None ---- Precision: 1.0 / Recall: 0.566 / Accuracy: 0.938
2019-06-05 17:40:38.640877


### Evaluating Random forest with GridSearchCV

In [0]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [0]:
from sklearn.model_selection import GridSearchCV

In [0]:
rf2 = RandomForestClassifier()
param = {'n_estimators': [10, 150, 300],
        'max_depth': [30, 60, 90, None]}

gs = GridSearchCV(rf2, param, cv=5, n_jobs=-1)

In [0]:
start = time.time()
print(datetime.datetime.now())

gs_fit1 = gs.fit(X_features, data['label'])
pd.DataFrame(gs_fit1.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 13:59:36.790276


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
8,34.293893,0.309868,0.361686,0.015720,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.978475,0.978437,0.974843,0.967655,0.972147,0.974313,0.004090,1
7,17.341426,0.206383,0.213321,0.006853,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.977578,0.977538,0.974843,0.968553,0.971249,0.973954,0.003557,2
11,36.007624,3.134525,0.370402,0.043921,None,300,"{'max_depth': None, 'n_estimators': 300}",0.977578,0.974843,0.975741,0.969452,0.970350,0.973594,0.003154,3
3,1.277291,0.010788,0.083406,0.003076,60,10,"{'max_depth': 60, 'n_estimators': 10}",0.975785,0.974843,0.973944,0.967655,0.968553,0.972157,0.003371,4
5,27.191136,0.335941,0.298861,0.013216,60,300,"{'max_depth': 60, 'n_estimators': 300}",0.976682,0.971249,0.972147,0.965858,0.971249,0.971439,0.003442,5


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

526.7089385986328
2019-06-05 14:08:23.499556


In [0]:
start = time.time()
print(datetime.datetime.now())

gs_fit2 = gs.fit(X_count_feat, data['label'])
pd.DataFrame(gs_fit2.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 14:08:29.598689


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
11,36.993969,2.278405,0.368309,0.049041,None,300,"{'max_depth': None, 'n_estimators': 300}",0.978475,0.973944,0.973944,0.967655,0.971249,0.973056,0.003561,1
8,34.959663,0.402545,0.373811,0.008417,90,300,"{'max_depth': 90, 'n_estimators': 300}",0.976682,0.974843,0.973046,0.968553,0.971249,0.972876,0.002819,2
10,19.612268,0.261646,0.253307,0.027661,None,150,"{'max_depth': None, 'n_estimators': 150}",0.978475,0.976640,0.973046,0.966757,0.969452,0.972876,0.004353,2
7,17.944235,0.220481,0.214960,0.008265,90,150,"{'max_depth': 90, 'n_estimators': 150}",0.979372,0.973944,0.973944,0.966757,0.969452,0.972696,0.004325,4
4,14.317859,0.236232,0.186498,0.008199,60,150,"{'max_depth': 60, 'n_estimators': 150}",0.975785,0.971249,0.970350,0.966757,0.969452,0.970720,0.002947,5


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

548.83469414711
2019-06-05 14:17:38.434192


### Exploring GradientBoostingClassifier Attributes & Hyperparameters

In [0]:
from sklearn.ensemble import GradientBoostingClassifier

In [0]:
print(dir(GradientBoostingClassifier))
print(GradientBoostingClassifier())

['_SUPPORTED_LOSS', '__abstractmethods__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_abc_cache', '_abc_negative_cache', '_abc_negative_cache_version', '_abc_registry', '_check_initialized', '_check_params', '_clear_state', '_estimator_type', '_fit_stage', '_fit_stages', '_get_param_names', '_get_tags', '_init_state', '_is_initialized', '_make_estimator', '_raw_predict', '_raw_predict_init', '_required_parameters', '_resize_state', '_staged_raw_predict', '_validate_estimator', '_validate_y', 'apply', 'decision_function', 'feature_importances_', 'fit', 'get_params', 'predict', 'predict_log_proba', 'predict_proba', 'score

### Building a Grid Search for Gradient Boosting

In [0]:
def train_GB(est, max_depth, lr):
    gb = GradientBoostingClassifier(n_estimators=est, max_depth=max_depth, learning_rate=lr)
    gb_model = gb.fit(X_train, y_train)
    y_pred2 = gb_model.predict(X_test)
    precision2, recall2, fscore2, train_support2 = score(y_test, y_pred2, pos_label='spam', average='binary')
    print('Est: {} / Depth: {} / LR: {} ---- Precision: {} / Recall: {} / Accuracy: {}'.format(
        est, max_depth, lr, round(precision2, 3), round(recall2, 3), 
        round((y_pred2==y_test).sum()/len(y_pred2), 3)))

In [0]:
start = time.time()
print(datetime.datetime.now())

for n_est in [50, 100, 150]:
    for max_depth in [3, 7, 11, 15]:
        for lr in [0.01, 0.1, 1]:
            train_GB(n_est, max_depth, lr)
            end = time.time()
            fit_time = (end - start)
            print(fit_time)
            print(datetime.datetime.now())
print(datetime.datetime.now())

2019-06-05 14:18:00.912576


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 3 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.849
36.93563795089722
2019-06-05 14:18:37.848230
Est: 50 / Depth: 3 / LR: 0.1 ---- Precision: 0.962 / Recall: 0.75 / Accuracy: 0.958
67.26325869560242
2019-06-05 14:19:08.176255
Est: 50 / Depth: 3 / LR: 1 ---- Precision: 0.88 / Recall: 0.827 / Accuracy: 0.957
89.08916139602661
2019-06-05 14:19:30.001743
Est: 50 / Depth: 7 / LR: 0.01 ---- Precision: 1.0 / Recall: 0.006 / Accuracy: 0.85
174.888503074646
2019-06-05 14:20:55.801526
Est: 50 / Depth: 7 / LR: 0.1 ---- Precision: 0.903 / Recall: 0.833 / Accuracy: 0.961
248.70944929122925
2019-06-05 14:22:09.622503
Est: 50 / Depth: 7 / LR: 1 ---- Precision: 0.87 / Recall: 0.833 / Accuracy: 0.956
299.46573209762573
2019-06-05 14:23:00.378773


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 11 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.849
427.7270152568817
2019-06-05 14:25:08.639863
Est: 50 / Depth: 11 / LR: 0.1 ---- Precision: 0.87 / Recall: 0.833 / Accuracy: 0.956
544.0622644424438
2019-06-05 14:27:04.974874
Est: 50 / Depth: 11 / LR: 1 ---- Precision: 0.868 / Recall: 0.863 / Accuracy: 0.96
587.7519927024841
2019-06-05 14:27:48.665045


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


Est: 50 / Depth: 15 / LR: 0.01 ---- Precision: 0.0 / Recall: 0.0 / Accuracy: 0.849
747.2563233375549
2019-06-05 14:30:28.168958
Est: 50 / Depth: 15 / LR: 0.1 ---- Precision: 0.892 / Recall: 0.839 / Accuracy: 0.961
908.9722263813019
2019-06-05 14:33:09.885452
Est: 50 / Depth: 15 / LR: 1 ---- Precision: 0.892 / Recall: 0.839 / Accuracy: 0.961
950.2841367721558
2019-06-05 14:33:51.197220
Est: 100 / Depth: 3 / LR: 0.01 ---- Precision: 0.978 / Recall: 0.53 / Accuracy: 0.927
1023.837482213974
2019-06-05 14:35:04.750480
Est: 100 / Depth: 3 / LR: 0.1 ---- Precision: 0.965 / Recall: 0.821 / Accuracy: 0.969
1078.3279728889465
2019-06-05 14:35:59.241010
Est: 100 / Depth: 3 / LR: 1 ---- Precision: 0.881 / Recall: 0.839 / Accuracy: 0.959
1120.7169239521027
2019-06-05 14:36:41.629535
Est: 100 / Depth: 7 / LR: 0.01 ---- Precision: 0.934 / Recall: 0.679 / Accuracy: 0.944
1295.494148015976
2019-06-05 14:39:36.407205
Est: 100 / Depth: 7 / LR: 0.1 ---- Precision: 0.907 / Recall: 0.869 / Accuracy: 0.967
1

### Evaluating Gradient Boosting with GridSearchCV

In [0]:
#from sklearn.model_selection import GridSearchCV

gb2 = GradientBoostingClassifier()
param = {
    'n_estimators': [100, 150], 
    'max_depth': [7, 11, 15],
    'learning_rate': [0.1]
}

clf = GridSearchCV(gb2, param, cv=5, n_jobs=-1)

In [0]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_features, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 17:41:45.049319


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
3,400.922848,4.832045,0.269422,0.029250,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.965022,0.978437,0.969452,0.969452,0.965858,0.969643,0.004756,1
5,532.916284,52.900317,0.270886,0.037375,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.966816,0.977538,0.966757,0.967655,0.968553,0.969463,0.004089,2
4,392.412370,5.954117,0.269900,0.020680,0.1,15,100,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.964126,0.975741,0.968553,0.969452,0.968553,0.969283,0.003726,3
1,252.042311,2.375593,0.230772,0.015467,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965919,0.978437,0.969452,0.965858,0.965858,0.969104,0.004866,4
2,284.396742,4.462068,0.257225,0.029042,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.963229,0.978437,0.968553,0.967655,0.964960,0.968565,0.005287,5


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

5619.923640966415
2019-06-05 19:15:24.973490


In [0]:
start = time.time()
print(datetime.datetime.now())

cv_fit = clf.fit(X_count_feat, data['label'])
pd.DataFrame(cv_fit.cv_results_).sort_values('mean_test_score', ascending=False)[0:5]

2019-06-05 19:15:24.987180


/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_learning_rate,param_max_depth,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
1,248.861480,2.559488,0.225899,0.004942,0.1,7,150,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.965022,0.980234,0.971249,0.961366,0.968553,0.969283,0.006405,1
5,518.763331,46.707888,0.256564,0.037313,0.1,15,150,"{'learning_rate': 0.1, 'max_depth': 15, 'n_est...",0.963229,0.979335,0.970350,0.963163,0.970350,0.969283,0.005957,1
3,389.536492,2.906899,0.239769,0.008859,0.1,11,150,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.963229,0.978437,0.968553,0.964960,0.970350,0.969104,0.005304,3
2,277.462814,2.945051,0.221269,0.004369,0.1,11,100,"{'learning_rate': 0.1, 'max_depth': 11, 'n_est...",0.964126,0.977538,0.966757,0.962264,0.970350,0.968205,0.005399,4
0,179.695558,1.012476,0.246364,0.035456,0.1,7,100,"{'learning_rate': 0.1, 'max_depth': 7, 'n_esti...",0.963229,0.977538,0.968553,0.962264,0.967655,0.967846,0.005421,5


In [0]:
end = time.time()
fit_time = (end - start)
print(fit_time)
print(datetime.datetime.now())

5366.802162647247
2019-06-05 20:44:51.789477
